In [161]:
import pandas as pd
import requests
import datetime

In [162]:
# Функция для получения данных:
def response_df(login, password):
    
    # Авторизация:
    param_request = {
        'json': '1',
        'login': login,
        'password': password
        }  

    response = requests.get('https://lk-gnezdo.com/cgi-bin/admin/auth.cgi', params=param_request)

    sid = response.json()['sid'] # session ID
    
    # Пустой датафрейм с параметрами для объединения данных
    data = { 
        "date": [],
        "id": [],
        "title": [],
        "url": [],
        "views_real": [],
        "views": [],
        "clicks": [],
        "money": []
    }
    df = pd.DataFrame(data)


    for i in range(20):  # число дней, за которые нужны данные от текущей даты. Если нужны по дням, а не за период 
        date = datetime.datetime.now() - datetime.timedelta(days = i)
        date = date.strftime("%Y-%m-%d")

        param_request = {
                'json': '1',
                'sid': sid,   # полученный session ID
                'mode': 'anons', # чтобы передавать данные по тизерам
                'date_start': date # если дата окончания не указана, она будет равна дате начала
                }  
        response = requests.get('http://news.gnezdo.ru/cgi-bin/admin/stat.cgi', params=param_request)


        data = response.json()
        data ["date"] = date

        df_ = pd.DataFrame(data)
        res = pd.concat([df_[x].apply(pd.Series).
                             dropna(how="all").reset_index(drop=True) for x in df_.columns],
                            axis=1, keys=df_.columns).dropna(how="all", axis=1)

        try:  # на случай если данных в конкретный день нет
            data_d = res['date'] # только дата
            data_d.set_axis(['date'], axis = 'columns', inplace = True) # переименование
            df_ = res['list']


            df_ = pd.merge(df_, data_d, left_index=True, right_index=True) # объединяю

            df_.rename(columns={'0': 'date'}, inplace=True) # переименование

            df = df.append(df_)
        except:
            pass
    
    return df

In [163]:
df1 = response_df('login1', 'password1')
df2 = response_df('login2', 'password2')

In [164]:
# Столбец для идентификации аккаунта:
df1 ['account'] = 'acc1'
df2 ['account'] = 'acc2'

In [165]:
df = pd.merge(df1, df2, how = 'outer') # объединение датафреймов

In [166]:
# Функции преобразования в формат google sheets:
def id_ (row):
    id_ = str(row['id']) 
    id_ = id_.replace('.', ',') 
    return id_

def money (row):
    money = str(row['money']) 
    money = money.replace('.', ',') 
    return money

# Применяю функции:
df['id'] = df.apply(id_, axis=1) 
df['money'] = df.apply(money, axis=1) 

df = df.sort_values(['account', 'date'], ascending=[True, True]) # сортировка по аккаунту и дате

In [168]:
# Запись в Google документ

import pygsheets

# Авторизация. Файл-ключ сгенерировал в google cloud
gc = pygsheets.authorize(service_file='innate-paratext.json')

# Открыть Гугл таблицу
sh = gc.open_by_key('you_doc_id')

# Выбрать лист
wks = sh.worksheet_by_title('gnezdo - import') # запись будет произведена в указанный лист

# Очистить лист, заданный диапазон
wks.clear(start='A1', end='T200000', fields='*')

# Обновить заданный выше лист
wks.set_dataframe(df,(1,1))